In [1]:
import os
from google.colab import drive

MAX_SEQ_LENGTH = 2048
D_TYPE = None
LOAD_IN_4BIT = True

SEED = 3407
COARSE_TRIALS = 12
COARSE_DATASET_SIZE = 1000
FINE_TRIALS = 4
FINE_DATASET_SIZE = 3000

BASE_MODEL_NAME = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
CHAT_TEMPLATE = "llama-3.1"
drive.mount('/content/drive')
CHECKPOINT_DIR = f'/content/drive/MyDrive/{BASE_MODEL_NAME}'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

Mounted at /content/drive


In [2]:
%%capture
!pip install unsloth optuna sentence-transformers rouge-score plotly
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

In [3]:
!pip uninstall wandb -y

Found existing installation: wandb 0.23.0
Uninstalling wandb-0.23.0:
  Successfully uninstalled wandb-0.23.0


In [4]:
from unsloth import FastLanguageModel
import torch
import optuna
import random
import gc
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only, standardize_sharegpt, get_chat_template
from datasets import load_dataset, concatenate_datasets

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Base Model

In [5]:
# Initial load for tokenizer and basic setup
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = D_TYPE,
    load_in_4bit = LOAD_IN_4BIT,
)

# Chat Template and Tokenizer Setup
tokenizer = get_chat_template(
    tokenizer,
    chat_template = CHAT_TEMPLATE,
)

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

## Data Preparations

In [6]:
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    # Ensure correct template application
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return { "text" : texts, }

# Data Loading and Formatting
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched=True, remove_columns=dataset.column_names)
dataset = dataset.select(range(50000))

# Create subset datasets for the two stages
coarse_dataset = dataset.shuffle(seed=SEED).select(range(COARSE_DATASET_SIZE))
fine_dataset = dataset.shuffle(seed=SEED).select(range(FINE_DATASET_SIZE))

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

## Hyperparameter Search

In [7]:
def objective(trial: optuna.Trial, training_dataset, trial_stage: str):
    gc.collect()
    torch.cuda.empty_cache()

    # Hyper-parameter space
    r_value = trial.suggest_categorical("r", [8, 16, 32])
    learning_rate = trial.suggest_float("learning_rate", 4e-5, 4e-4, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.005, 0.05, log=True)

    print(f"\n--- Starting Trial {trial.number} ({trial_stage} stage) ---")
    print(f"Params: LR={learning_rate:.2e}, r={r_value}, weight_decay={weight_decay}, Data Size: {len(training_dataset)}")

    # 2. Model Setup
    current_base_model, current_tokenizer = FastLanguageModel.from_pretrained(
        model_name = BASE_MODEL_NAME,
        max_seq_length = MAX_SEQ_LENGTH,
        dtype = D_TYPE,
        load_in_4bit = LOAD_IN_4BIT,
    )
    current_tokenizer = get_chat_template(current_tokenizer, chat_template=CHAT_TEMPLATE)


    model = FastLanguageModel.get_peft_model(
        current_base_model,
        r=r_value,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_alpha=r_value,
        lora_dropout=0,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=SEED,
    )

    # 3. Training Arguments Setup
    trial_output_dir = os.path.join(CHECKPOINT_DIR, f"{trial_stage}_trial_{trial.number}")

    args = TrainingArguments(
        output_dir=trial_output_dir,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=1,
        learning_rate=learning_rate,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=5,
        optim="adamw_8bit",
        weight_decay=weight_decay,
        lr_scheduler_type="linear",
        seed=SEED,
        report_to="none",
        save_strategy='no', # Only evaluating loss, no need to save
    )

    # 4. SFTTrainer Initialization and Training
    trainer = SFTTrainer(
        model=model,
        tokenizer=current_tokenizer,
        train_dataset=training_dataset,
        eval_dataset=training_dataset,
        dataset_text_field="text",
        max_seq_length=MAX_SEQ_LENGTH,
        data_collator=DataCollatorForSeq2Seq(tokenizer=current_tokenizer),
        dataset_num_proc=6,
        packing=False,
        args=args,
    )

    trainer = train_on_responses_only(
        trainer,
        instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
        response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
    )

    trainer.train()

    # 5. Evaluation and Cleanup
    final_eval_loss = trainer.evaluate()["eval_loss"]

    # Clean up memory
    del model, current_base_model, trainer, current_tokenizer
    gc.collect()
    torch.cuda.empty_cache()

    return final_eval_loss



### Coarse-grained Search

In [8]:
coarse_study = optuna.create_study(
    direction="minimize",
    study_name="coarse-hyper-params",
    storage=f"sqlite:///{CHECKPOINT_DIR}/optuna_study.db",
    load_if_exists=True,
)

completed_coarse = [
    t for t in coarse_study.trials
    if t.state == optuna.trial.TrialState.COMPLETE
]
coarse_needed = COARSE_TRIALS - len(completed_coarse)

print("\n" + "=" * 60)
print(f"STAGE 1: COARSE SEARCH ({len(completed_coarse)}/{COARSE_TRIALS} done)")
print("=" * 60)

if coarse_needed > 0:
    coarse_study.optimize(
        lambda trial: objective(trial, coarse_dataset, "coarse"),
        n_trials=coarse_needed,
        show_progress_bar=True
    )
else:
    print("Coarse search already complete.")

[I 2025-12-06 05:07:42,746] Using an existing study with name 'coarse-hyper-params' instead of creating a new one.



STAGE 1: COARSE SEARCH (12/12 done)
Coarse search already complete.


In [9]:

from plotly.io import show
fig = optuna.visualization.plot_contour(coarse_study)
show(fig)

In [10]:
fig = optuna.visualization.plot_param_importances(coarse_study)
show(fig)

### Fine-grained Search

In [11]:
fine_study = optuna.create_study(
    direction="minimize",
    study_name="fine-hyper-params",
    storage=f"sqlite:///{CHECKPOINT_DIR}/optuna_study.db",
    load_if_exists=True,
)

coarse_trials = coarse_study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE])
best_coarse_trials = sorted(coarse_trials, key=lambda t: t.value)[:FINE_TRIALS]

print("\n" + "=" * 60)
print(f"STAGE 2: FINE SEARCH (Top {len(best_coarse_trials)} Coarse Candidates)")
print("=" * 60)

# 2. Iterate through best candidates and run them ONE BY ONE if missing
for coarse_trial in best_coarse_trials:

    # Check if a COMPLETED fine trial already exists for this specific coarse_trial ID
    already_done = False
    for t in fine_study.trials:
        if (t.state == optuna.trial.TrialState.COMPLETE
            and t.user_attrs.get("source_coarse_id") == coarse_trial.number):
            already_done = True
            break

    if already_done:
        print(f"Skipping Fine Tune for Coarse Trial #{coarse_trial.number} (Already Complete)")
        continue

    # If we are here, we need to run this specific trial
    print(f"Running Fine Tune for Coarse Trial #{coarse_trial.number}...")

    # Enqueue the parameters
    fine_study.enqueue_trial(
        params=coarse_trial.params,
        user_attrs={
            "source_coarse_id": coarse_trial.number
        }
    )

    # Run exactly ONE trial to process the queue item we just added
    fine_study.optimize(
        lambda trial: objective(trial, fine_dataset, "fine"),
        n_trials=1,
        show_progress_bar=False
    )

# Final Results
fine_trials = [
    t for t in fine_study.trials
    if t.state == optuna.trial.TrialState.COMPLETE
]

if fine_trials:
    best_fine = min(fine_trials, key=lambda t: t.value)
    print("\n🏆 Best Result:")
    print(f"Loss: {best_fine.value:.4f}")
    print(f"Params: {best_fine.params}")

[I 2025-12-06 05:07:46,083] Using an existing study with name 'fine-hyper-params' instead of creating a new one.



STAGE 2: FINE SEARCH (Top 4 Coarse Candidates)
Skipping Fine Tune for Coarse Trial #1 (Already Complete)
Skipping Fine Tune for Coarse Trial #12 (Already Complete)
Skipping Fine Tune for Coarse Trial #3 (Already Complete)
Skipping Fine Tune for Coarse Trial #0 (Already Complete)

🏆 Best Result:
Loss: 0.7303
Params: {'r': 32, 'learning_rate': 0.0002475732755729107, 'weight_decay': 0.016946784233875283}


## Training

In [12]:
import os
import time
import gc
import torch

from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only

gc.collect()
torch.cuda.empty_cache()

r_value = best_fine.params["r"]
weight_decay = best_fine.params["weight_decay"]
learning_rate = best_fine.params["learning_rate"]

current_base_model, current_tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = D_TYPE,
    load_in_4bit = LOAD_IN_4BIT,
)
current_tokenizer = get_chat_template(current_tokenizer, chat_template=CHAT_TEMPLATE)

model = FastLanguageModel.get_peft_model(
    current_base_model,
    r=r_value,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=r_value,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=SEED,
)


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    eval_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=10,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=8,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        learning_rate=learning_rate,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=20,
        optim="adamw_8bit",
        weight_decay=weight_decay,
        lr_scheduler_type="linear",
        seed=SEED,
        num_train_epochs=1,
        save_strategy='steps',
        save_steps=100,
        save_total_limit=3,
        output_dir=CHECKPOINT_DIR,
        gradient_checkpointing=True,
        report_to=None,
    ),
)

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

checkpoints = [f for f in os.listdir(CHECKPOINT_DIR) if f.startswith('checkpoint')]
checkpoints.sort(key=lambda x: int(x.split('-')[-1]))
if checkpoints:
    latest_checkpoint = os.path.join(CHECKPOINT_DIR, checkpoints[-1])
    print('Resuming from:', latest_checkpoint)
else:
    latest_checkpoint = None
    print('No checkpoint found. Starting fresh.')
trainer_stats = trainer.train(resume_from_checkpoint=latest_checkpoint)

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.11.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/50000 [00:00<?, ? examples/s]

Map (num_proc=6):   0%|          | 0/50000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Resuming from: /content/drive/MyDrive/unsloth/Llama-3.2-3B-Instruct-bnb-4bit/checkpoint-1563


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 1 | Total steps = 1,563
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 48,627,712 of 3,261,377,536 (1.49% trained)


Step,Training Loss


In [13]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.184 GB of memory reserved.


In [14]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

0.0295 seconds used for training.
0.0 minutes used for training.
Peak reserved memory = 5.184 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 35.167 %.
Peak reserved memory for training % of max memory = 0.0 %.


## Evaluation

In [15]:
# Original prompts, expected answer, modified prompt
prompts = [
    (
        "Discuss the efforts of the United Nations to advance world peace.",
        "The United Nations is a global organization with a mission to promote international peace and security. Since its founding in 1945, the United Nations has been a major force in global diplomacy, working to resolve conflicts both on a regional and international level. The United Nations has organized peacekeeping missions and negotiated ceasefires, arms embargoes and disarmament agreements in countries around the world. The United Nations also works to provide humanitarian and economic assistance to areas affected by war, famine, and natural disasters. The United Nations also seeks to promote human rights and gender equality, uphold international law and advocate for global economic and social justice. By providing a forum for dialogue and discussion between member states, the United Nations seeks to advance dialogue and constructively address issues that have the potential to lead to conflict and war.",
        "Describe the United Nations’ efforts aimed at promoting global peace."
    ),
    (
        "Why is a wing necessary for flight, given that an engine can provide the necessary energy to counter gravity and propel an airplane forward?",
        "The wing serves a crucial role in flight efficiency by converting the engine's horizontal energy into vertical lift without significantly increasing the overall energy expenditure. While tilting the engine downward could create lift, it would also cause a significant loss in forward thrust due to the conservation of energy and momentum.\n\nTo understand this better, consider the trade-off between momentum and energy. The wing generates lift by moving a large volume of air a small distance (or a small volume of air a large distance), which results in lower kinetic energy transfer to the air. This is more energy-efficient than directly tilting the engine, which would create a smaller volume of air moving at a higher velocity, dissipating more energy as noise and heat.\n\nAdditionally, the wing's shape and size enable it to optimize this trade-off, with long, thin wings being particularly efficient at generating lift with minimal energy loss. In contrast, using the engine alone to produce both lift and thrust would be less efficient, as it would require a significant amount of energy to compress air, mix it with fuel, and accelerate the resulting gases, all of which contribute to energy wastage.\n\nIn essence, the wing acts as a mediator that redirects the engine's energy into both horizontal thrust and vertical lift, ensuring a more efficient flight experience. This is why wings are essential in aircraft design, even when engines can provide the necessary energy for lift and propulsion.",
        "Why are wings essential for flight if engines can provide necessary energy for countering gravity and thrusting the plane forward?"
    ),
    (
        "How can I plot a circle with matplotlib.pyplot in Python, taking as input center (x,y) and radius r? I attempted several variations similar to the code below but have not been successful. Here is the code I tried:\n\n```python\nimport matplotlib.pyplot as plt\ncircle=plt.Circle((0,0),2)\n# here must be something like circle.plot() or not?\nplt.show()\n```\nI am specifically looking for a solution that does not involve the use of pylab.",
        "The circle object that you created using `plt.Circle` is a patch, and must be added to the current axes using the `add_patch` method. The following code shows how this can be done:\n\n```python\nimport matplotlib.pyplot as plt\n\nfig, ax = plt.subplots() # note we must use plt.subplots, not plt.subplot\n# (or if you have an existing figure)\n# fig = plt.gcf()\n# ax = fig.gca()\n\ncircle = plt.Circle((0, 0), 2, fill = False)\n\nax.add_patch(circle)\n\n# Set the limits of the plot to be equal, so the circle isn't distorted\nax.axis('equal')\n\nplt.show()\n```\n\nIn the above code:\n- `plt.subplots()` is used to create a new figure and a set of subplots.\n- `plt.Circle((0, 0), 2, fill = False)` creates a circle centered at (0, 0) with radius 2. `fill = False` makes the circle empty. If you want a filled circle, you can omit this parameter or set it to `True`.\n- `ax.add_patch(circle)` adds the circle to the axes.\n- `ax.axis('equal')` sets the aspect of the plot box to 'equal'. The aspect is the ratio of the y-unit to the x-unit and by setting it to 'equal', the units on the x and y axes are the same, ensuring that the circle is not distorted.",
        "How can I draw a circle in matplotlib using only its center (x, y) and radius r? Several similar variants to the attempt below have not been successful. This is my code:```python\nimport matplotlib.pyplot as plt\ncircle=plt.Circle((1,0),5)\n# Something like circle.plot() here?\nplt.show()\n```\nI am not looking for an implementation that uses pylab."
    ),
    (
        "Discuss two practical and cost-effective ways to help the environment, one of which must involve implementing renewable energy sources, while the other must include reducing carbon emissions from transportation, and explain how these solutions can be implemented in developing countries with limited resources. Additionally, provide a comparative analysis of the benefits and drawbacks of these solutions in terms of their impact on economic growth, social equity, and environmental sustainability. Finally, outline a comprehensive action plan for governments, NGOs, and individuals to promote these solutions at the local, national, and international levels, taking into account the cultural, political, and technological factors that may affect their success. Consider providing case studies or real-life examples to illustrate the potential of these solutions and inspire others to take action.",
        "Solution 1: Implementing Renewable Energy Sources\nOne practical and cost-effective way to help the environment is to implement renewable energy sources. Renewable energy sources such as solar, wind, and hydroelectric power can be harnessed to generate electricity without emitting harmful greenhouse gases. This can help reduce carbon emissions and mitigate the impact of climate change.\nIn developing countries with limited resources, implementing renewable energy sources can be challenging due to the high cost of technology and infrastructure. However, there are several ways to overcome these challenges. One approach is to promote decentralized renewable energy systems that can be installed in remote areas without access to the main power grid. This can be done through microfinance schemes that provide loans to households and small businesses to purchase renewable energy systems.\nAnother approach is to promote public-private partnerships that can help mobilize resources and expertise to develop renewable energy projects. Governments can provide incentives such as tax breaks and subsidies to attract private sector investment in renewable energy projects. NGOs can also play a role in promoting renewable energy by providing technical assistance and capacity building to local communities.\nIn terms of economic growth, implementing renewable energy sources can create new job opportunities and stimulate economic development. However, there may be initial costs associated with building the necessary infrastructure, which could be a challenge for countries with limited resources. Additionally, there may be social equity concerns if the benefits of renewable energy are not distributed evenly among different groups.\nSolution 2: Reducing Carbon Emissions from Transportation\nAnother practical and cost-effective way to help the environment is to reduce carbon emissions from transportation. This can be done by promoting the use of public transportation, encouraging the adoption of electric vehicles, and promoting non-motorized transportation such as walking and cycling.\nIn developing countries with limited resources, reducing carbon emissions from transportation can be challenging due to the lack of infrastructure and resources. However, there are several ways to overcome these challenges. One approach is to promote the use of public transportation by investing in bus rapid transit systems and promoting the use of shared transportation services such as carpooling and bike-sharing.\nAnother approach is to promote the adoption of electric vehicles by providing incentives such as tax breaks and subsidies to encourage their purchase. Governments can also invest in charging infrastructure to support the adoption of electric vehicles.\nIn terms of economic growth, reducing carbon emissions from transportation can create new job opportunities in the renewable energy and transportation sectors. However, there may be initial costs associated with building the necessary infrastructure, which could be a challenge for countries with limited resources. Additionally, there may be social equity concerns if the benefits of reduced carbon emissions are not distributed evenly among different groups.\nComprehensive Action Plan\nTo promote these solutions at the local, national, and international levels, governments, NGOs, and individuals can take the following actions:\n1. Raise awareness: Governments and NGOs can launch awareness campaigns to educate the public about the benefits of renewable energy and reducing carbon emissions from transportation.\n2. Provide incentives: Governments can provide tax breaks, subsidies, and other incentives to promote the adoption of renewable energy and electric vehicles.\n3. Build infrastructure: Governments can invest in the necessary infrastructure to support renewable energy and public transportation systems.\n4. Promote public-private partnerships: Governments can work with the private sector to mobilize resources and expertise to develop renewable energy projects and promote sustainable transportation.\n5. Encourage behavior change: Individuals can be encouraged to adopt sustainable transportation habits such as walking, cycling, and carpooling.\n6. Support research and development: Governments and NGOs can invest in research and development to improve the efficiency and affordability of renewable energy and sustainable transportation technologies.\nCase Study: India\nIndia is a developing country that has implemented several initiatives to promote renewable energy and sustainable transportation. The government has set a target of achieving 175 GW of renewable energy capacity by 2022, which includes 100 GW of solar power. The government has also launched several initiatives to promote electric vehicles, including tax breaks and subsidies for their purchase. Additionally, the government has invested in public transportation infrastructure, including the development of bus rapid transit systems in major cities.\nConclusion\nImplementing renewable energy sources and reducing carbon emissions from transportation are two practical and cost-effective ways to help the environment. These solutions can be implemented in developing countries with limited resources through a combination of public-private partnerships, incentives, and infrastructure development. While there may be initial costs associated with these solutions, they can stimulate economic growth, promote social equity, and contribute to environmental sustainability. Governments, NGOs, and individuals can take a comprehensive approach to promote these solutions at the local, national, and international levels, taking into account the cultural, political, and technological factors that may affect their success.",
        "Examine two practical and affordable strategies for environmental protection—one focusing on implementing renewable energy technologies, and the other targeting the reduction of carbon emissions from transportation. Detail how these approaches can be applied in resource-constrained developing nations. Furthermore, offer a comparative analysis of their advantages and disadvantages regarding economic growth, social fairness, and ecological sustainability. Conclude by proposing a detailed action plan for governments, NGOs, and individuals to advance these initiatives across local, national, and global arenas, while accounting for cultural, political, and technological variables that could influence their success. Where possible, include case studies or real-life examples to demonstrate the potential of these methods and motivate others to make a difference."
    ),
    (
        "Explain the chain of events leading to the signing of the Magna Carta in 1215.",
        "1. King John's rule: King John of England faced numerous problems, including failed military campaigns, high taxation, and conflicts with the Pope.2. Barons' discontent: English barons were dissatisfied with King John's rule due to heavy taxes, loss of influence, and disregard for their rights and privileges.3. Tensions escalate: In 1215, rebel barons captured London, effectively forcing King John into negotiations.4. Magna Carta negotiations: Barons presented King John with a document outlining their demands, which aimed to limit the king's power by protecting their rights and ensuring fair treatment.5. Signing of the Magna Carta: On June 15, 1215, King John signed the Magna Carta at Runnymede, ultimately symbolizing the first significant step towards limiting the power of the monarchy and establishing the foundation for future democratic principles.",
        "Outline the sequence of events that led to the creation of the Magna Carta in 1215."
    ),
    (
        "How is stellar parallax measured?",
        "Stellar parallax is measured by observing the apparent shift in the position of a star as seen from two different points on Earth's orbit around the Sun. This shift is caused by the change in the angle at which the star is viewed from these two points. The distance to the star can then be calculated using trigonometry.\n\nThe distance to a star is measured in parsecs. One parsec is equal to 3.26 light-years. The parallax angle is measured in arcseconds. One arcsecond is equal to 1/3600 of a degree.\n\nThe parallax angle is very small, even for the nearest stars. The parallax angle of the nearest star, Alpha Centauri, is only 0.772 arcseconds. This means that Alpha Centauri is about 4.3 light-years away from Earth.\n\nStellar parallax is a very important tool for astronomers. It allows them to measure the distances to stars and other objects in space. This information is essential for understanding the structure and evolution of the universe.",
        "How does one measure stellar parallax?"
    ),
    (
        "What is bacterial transformation, and how does it occur?",
        "Bacterial transformation is a genetic modification process where bacteria take up external DNA from their environment and incorporate it into their own genome. This can result in the acquisition of new traits or characteristics.\n\nHere's a step-by-step explanation:\n1. **DNA Preparation**: The gene or genetic material of interest is extracted from its original source and cut using a specific restriction enzyme.\n2. **Plasmid Incorporation**: This DNA fragment is then inserted into a plasmid, a small, circular DNA molecule that is separate from the bacterial chromosome and can replicate independently.\n3. **Ligation**: The plasmid with the inserted DNA is ligated, or joined together, to create a recombinant plasmid.\n4. **Transformation**: The recombinant plasmid is introduced into bacteria, often using a laboratory procedure to facilitate uptake. Naturally transformable bacteria can also take up DNA spontaneously under certain environmental conditions.\n5. **Integration**: Once inside the bacterial cell, the plasmid with the foreign DNA integrates into the bacterial chromosome or remains extrachromosomal, and the new genetic information is expressed.\n\nThis process is widely used in molecular biology for gene cloning and genetic engineering applications.",
        "What is bacterial transformation, and through what process does it take place?"
    ),
    (
        "What triggers the release of neurotransmitters?",
        "The axon is a long, thin projection that extends from the cell body of a neuron. It is responsible for transmitting electrical signals away from the cell body. When an electrical signal reaches the end of the axon, it triggers the release of neurotransmitters into the synaptic cleft, the space between two neurons.\n\nNeurotransmitters are chemical messengers that allow neurons to communicate with each other. They are stored in small vesicles at the end of the axon. When an electrical signal reaches the end of the axon, it causes the vesicles to fuse with the cell membrane and release their neurotransmitters into the synaptic cleft.\n\nThe neurotransmitters then diffuse across the synaptic cleft and bind to receptors on the dendrites of the postsynaptic neuron. This binding triggers a new electrical signal in the postsynaptic neuron, which can then be transmitted to other neurons.\n\nThe release of neurotransmitters is a critical step in the process of neural communication. It allows neurons to send signals to each other and to control a wide range of bodily functions, including movement, thought, and emotion.\n####\nNeurotransmitters are released at the presynaptic terminal in response to a threshold action potential or graded electrical potential.",
        "What causes neurons to release neurotransmitters?"
    ),
    (
        "How does a processor execute an assembly instruction, like \"ADD 1,2,\" and produce an output, in the context of mapping a high-level block diagram to a circuit diagram?",
        "To understand how a processor performs the addition of two numbers, such as 1 and 2, at a lower level, you need to delve into the underlying architecture and operations. Here's a simplified overview:\n\n1. **Language Abstraction**: You're familiar with high-level programming languages, but assembly language is one step lower. It directly corresponds to specific machine instructions, like \"ADD 1,2,\" which tells the processor to add the numbers stored at memory addresses 1 and 2.\n\n2. **Processor Components**: A processor contains functional units, like the arithmetic logic unit (ALU), which performs arithmetic operations. The control unit manages the flow of data and instruction execution.\n\n3. **数字表示**: In a computer, numbers are typically represented in 2's complement notation. So, 1 and 2 would be binary numbers, and their sum would involve binary addition.\n\n4. **Circuit-Level Operations**: At the circuit level, an ALU is built using logic gates (such as AND, OR, and NOT gates) that implement the addition operation. A \"Full Adder\" circuit is used for this purpose, handling carry-in and carry-out for binary digits.\n\n5. **Execution**: When the \"ADD 1,2\" instruction is executed, the processor fetches the numbers from memory, loads them into registers, and sends them to the ALU. The ALU performs the binary addition, generates the sum (3 in this case), and stores it back in a register or memory.\n\nFor a deeper understanding, consider resources on digital electronics, particularly CMOS logic and binary arithmetic. A recommended course for beginners is the Georgia Tech ECE2030 class, which provides an introduction to computer engineering, though the course materials might be more detailed than what you're seeking.\n\nRemember, understanding this process requires a willingness to learn about the layers of abstraction beneath the software level. Good luck on your journey!",
        "How does a processor turn an assembly instruction like \"ADD 1,2\" into an actual hardware-level operation and produce an output?"
    ),
    (
        "How do protons overcome their repulsion and come close enough to form a nucleus, considering the short range of nuclear forces?",
        "When protons are initially far apart, they possess sufficient kinetic energy to approach each other despite the electrostatic repulsion between them. As they get closer, they can experience a quantum mechanical phenomenon called tunneling, which allows them to pass through the energy barrier that would normally keep them apart due to their repulsion. Once they tunnel through this barrier, the strong nuclear force becomes significant, and the protons can be held together within a nucleus.\n\nIn some cases, this results in a temporary diproton nucleus, which is unstable. A process called beta decay occurs, where one of the protons converts into a neutron, forming a stable deuterium nucleus (deuteron). This deuteron can then participate in further nuclear fusion reactions, like those that power the sun.",
        "How do protons manage to overcome their mutual repulsion and get close enough to make a nucleus, given the nuclear force’s short range?"
    ),
    (
        "Develop an all-encompassing strategy that takes into account various factors such as economic, social, and environmental impacts to encourage the adoption of sustainable agricultural methods in densely populated urban regions. Additionally, devise ways to guarantee food availability and affordability for low-income neighborhoods while addressing the potential challenges that may arise in implementing such a plan.",
        "To encourage the adoption of sustainable agricultural methods in densely populated urban regions, we need to consider economic, social, and environmental factors. Here is an all-encompassing strategy that can help:\n1. Provide education and training: Educate farmers and urban residents about sustainable agricultural practices, such as organic farming, permaculture, and hydroponics. Train them on how to use these methods effectively to increase yields and reduce costs.\n2. Create incentives: Offer tax incentives, subsidies, and grants to farmers who adopt sustainable practices. This will encourage more farmers to switch to sustainable methods and help offset the costs of transitioning.\n3. Promote community gardens: Encourage the establishment of community gardens in urban areas. These gardens can provide fresh produce to low-income neighborhoods and promote social cohesion.\n4. Partner with local businesses: Partner with local businesses to create a market for sustainable produce. This can help ensure that farmers have a reliable source of income and that consumers have access to fresh, healthy food.\n5. Address food waste: Develop programs to reduce food waste by promoting composting and recycling. This can help reduce the environmental impact of agriculture and provide a source of fertilizer for sustainable farming practices.\n6. Address food deserts: Work to ensure that low-income neighborhoods have access to fresh, healthy food. This can be done by promoting farmers' markets, community gardens, and other initiatives that increase access to fresh produce.\n7. Address transportation issues: Develop transportation networks to ensure that produce can be transported from rural areas to urban areas quickly and efficiently. This can help reduce the environmental impact of transportation and ensure that produce remains fresh and affordable.\n8. Address water scarcity: Develop water conservation programs to ensure that sustainable agriculture practices are not hindered by water scarcity. This can include rainwater harvesting, drip irrigation, and other water conservation methods.\n9. Address energy needs: Develop renewable energy programs to power sustainable agriculture practices. This can include solar and wind power, as well as other renewable energy sources.\n10. Address land use: Develop zoning laws and land use policies that encourage sustainable agriculture practices. This can include urban agriculture zoning, land trusts, and other policies that promote sustainable land use.\nIn implementing this plan, we need to address potential challenges such as resistance from farmers, lack of funding, and limited resources. To guarantee food availability and affordability for low-income neighborhoods, we need to ensure that sustainable agriculture practices are economically viable and that produce is accessible to all. This can be done by partnering with local businesses, developing transportation networks, and addressing food deserts. By taking a holistic approach that addresses economic, social, and environmental factors, we can encourage the adoption of sustainable agriculture practices in densely populated urban regions.",
        "Create a comprehensive strategy that considers economic, social, and environmental impacts to promote sustainable agriculture in highly populated urban areas. Also, design strategies to ensure food accessibility and affordability in low-income communities, while tackling the potential obstacles that could emerge during the plan’s implementation."
    ),
    (
        "Discuss the origin and evolution of circuses, touching on significant milestones and changes in the types of acts performed throughout history.",
        "The origin of circuses can be traced back to ancient Rome, where the first recorded circus-like event took place in 6th century BCE. The Circus Maximus, a large stadium built for chariot racing and other spectacles, was the center of Roman entertainment. However, these early events were not what we consider as modern circuses today; they mainly focused on horse and chariot races, athletic competitions, and staged battles.\n\nThe modern concept of the circus began to take shape in the late 18th century when Englishman Philip Astley, a former cavalry sergeant major, established the first permanent circular arena or \"ring\" for equestrian performances in London in 1768. He combined daring horsemanship with acrobatics, clowning, and music, creating a new form of popular entertainment that would become known as the circus.\n\nIn the early 19th century, the circus spread across Europe and North America, evolving into various forms and incorporating different acts. Tightrope walkers, jugglers, trapeze artists, and animal trainers became regular features of the circus, alongside equestrian acts and clowns. One significant milestone was the introduction of the circus tent by American showman Joshua Purdy Brown in the 1820s, which allowed circuses to travel more easily and perform in temporary venues.\n\nP.T. Barnum is another key figure in the evolution of the circus. In the mid-19th century, he founded the famous Barnum & Bailey Circus, which eventually merged with the Ringling Brothers Circus to create the largest traveling circus in the United States. Barnum's circus featured elaborate costumes, exotic animals, and human oddities, making it a spectacle unlike anything seen before.\n\nThroughout the 20th century, the circus continued to evolve, adapting to changing tastes and technologies. Traditional animal acts began to decline due to growing concerns about animal welfare, leading to an increased focus on human performances. Cirque du Soleil, founded in 1984 by Guy Laliberté and Daniel Gauthier, is a prime example of this shift. The Canadian company revolutionized the circus industry with its innovative blend of acrobatics, dance, music, and theatrical elements, creating a more artistic and sophisticated form of entertainment.\n\nIn recent years, there has been a resurgence of interest in smaller, more intimate circuses that emphasize skilled human performers and unique acts. These contemporary circuses often incorporate elements of street performance, cabaret, and live music, offering an alternative to the large-scale productions of traditional circuses.\n\nIn conclusion, the origin and evolution of circuses have seen significant milestones and changes in the types of acts performed throughout history. From ancient Roman spectacles to modern-day artistic performances, the circus has continually adapted to reflect societal values and entertain audiences worldwide.",
        "Describe how circuses originated and evolved, outlining important milestones and how their types of acts have changed throughout history."
    ),
    (
        "Given the sentence \"Three men go for a ride in a boat.\" is it true that \"The three men are on a large sailboat.\"?\nLet's solve step-by-step:",
        "Step 1: Analyze the given sentence\nThe given sentence is \"Three men go for a ride in a boat.\" This tells us that there are three men and they are going for a ride in a boat. However, the type and size of the boat are not specified.\n\nStep 2: Compare the given sentence with the statement\nThe statement is \"The three men are on a large sailboat.\" This statement provides additional information about the boat - that it is a large sailboat.\n\nStep 3: Determine if the statement is true based on the given sentence\nSince the given sentence does not specify the type or size of the boat, we cannot determine if the statement is true or false based on the information provided. There is not enough information in the given sentence to confirm that the three men are on a large sailboat. \n\nConclusion:\nBased on the given sentence, we cannot conclude that \"The three men are on a large sailboat.\" There is not enough information provided to determine the truth of the statement.",
        "Given the sentence \"Three men go for a ride in a boat,\" can we conclude they are on a large sailboat?. Solve step-by-step:"
    ),
    (
       "What are the forces responsible for the Earth's orbit around the Sun, and how do they interact?",
       "The two forces responsible for the Earth's orbit around the Sun are gravitational force and centrifugal force. Gravitational force, according to the Law of Universal Gravity, attracts the Earth towards the Sun. Centrifugal force, a fictitious force, acts outwards from the Sun. These two forces balance each other, resulting in the Earth's stable orbit around the Sun.",
       "Which forces shape the Earth's orbit around the Sun, and what are their interactions?"
    ),
]

In [16]:
import pandas as pd
import csv
from sentence_transformers import SentenceTransformer
import torch
from rouge_score import rouge_scorer


# Initialize S-BERT model once
# 'all-MiniLM-L6-v2' is a common, fast, and effective choice for general English text
SBERT_MODEL = SentenceTransformer("all-MiniLM-L6-v2")


def calculate_sbert_cosine_similarity(text1, text2, model=SBERT_MODEL):
    """
    Calculates Cosine Similarity between two strings using Sentence-BERT embeddings.
    Returns score between -1 and 1, where 1.0 means perfect semantic similarity.
    """
    if not text1 or not text2:
        return 0.0

    # Generate dense vector embeddings for both texts
    embeddings1 = model.encode(text1)
    embeddings2 = model.encode(text2)

    # Compute cosine similarities
    similarities = model.similarity(embeddings1, embeddings2)
    return float(similarities[0][0])

def calculate_rouge_l_recall(reference, hypothesis):
    """
    Calculates ROUGE-L Recall score.
    Returns score between 0 and 1, where 1.0 means perfect recall.
    """
    if not reference or not hypothesis:
        return 0.0

    # Use use_stemmer=True for better matching
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)

    # Extract the Recall score
    return scores['rougeL'].recall

def evaluate(model, tokenizer, prompts, csv_path):
    outputs = []

    for original_prompt, expected_answer, modified_prompt in prompts:
        messages = [{"role": "user", "content": modified_prompt}]

        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda")

        output_tokens = model.generate(
            input_ids=inputs,
            temperature=1.5,
            min_p=0.1,
            max_new_tokens=2048,
        )

        decoded_output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
        assistant_start = decoded_output.find("assistant\n\n")
        assistant_output = decoded_output[assistant_start + len("assistant\n\n"):].strip()

        # --- Metric Calculation ---

        # 1. Semantic Similarity (Sentence-BERT Cosine Sim)
        sbert_cosine_sim = calculate_sbert_cosine_similarity(
            expected_answer,
            assistant_output
        )

        # 2. Content Coverage (ROUGE-L Recall)
        rouge_l_recall = calculate_rouge_l_recall(
            expected_answer,
            assistant_output
        )

        outputs.append({
            "original_prompt": original_prompt,
            "expected_answer": expected_answer,
            "modified_prompt": modified_prompt,
            "actual_output": assistant_output,
            "sbert_cosine_sim": sbert_cosine_sim,
            "rouge_l_recall": rouge_l_recall,
        })

    df = pd.DataFrame(outputs)

    # Clean strings before saving to CSV
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).str.replace('\n', '\\n').str.replace('\r', '\\r')

    df.to_csv(csv_path, index=False, quoting=csv.QUOTE_ALL)

    return df

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
%%capture
base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = D_TYPE,
    load_in_4bit = LOAD_IN_4BIT,
)
FastLanguageModel.for_inference(base_model)
FastLanguageModel.for_inference(model)

In [18]:
evaluate(base_model, base_tokenizer, prompts, f"{CHECKPOINT_DIR}/base_eval.csv")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


,original_prompt,expected_answer,modified_prompt,actual_output,sbert_cosine_sim,rouge_l_recall
0,Discuss the efforts of the United Nations to a...,The United Nations is a global organization wi...,Describe the United Nations’ efforts aimed at ...,The United Nations (UN) has made significant e...,0.753761,0.304348
1,"Why is a wing necessary for flight, given that...",The wing serves a crucial role in flight effic...,Why are wings essential for flight if engines ...,Although engines provide the necessary energy ...,0.675520,0.246964
2,How can I plot a circle with matplotlib.pyplot...,The circle object that you created using `plt....,How can I draw a circle in matplotlib using on...,Here's a function to plot a circle using matpl...,0.721404,0.228972
3,Discuss two practical and cost-effective ways ...,Solution 1: Implementing Renewable Energy Sour...,Examine two practical and affordable strategie...,**Strategy 1: Promoting Renewable Energy Techn...,0.724846,0.210396
4,Explain the chain of events leading to the sig...,1. King John's rule: King John of England face...,Outline the sequence of events that led to the...,Here is the outline of events that led to the ...,0.799022,0.270677
5,How is stellar parallax measured?,Stellar parallax is measured by observing the ...,How does one measure stellar parallax?,Measuring stellar parallax is a fascinating te...,0.831019,0.289941
6,"What is bacterial transformation, and how does...",Bacterial transformation is a genetic modifica...,"What is bacterial transformation, and through ...",Bacterial transformation is a process by which...,0.894761,0.262295
7,What triggers the release of neurotransmitters?,"The axon is a long, thin projection that exten...",What causes neurons to release neurotransmitters?,Neurons release neurotransmitters in response ...,0.666789,0.202020
8,How does a processor execute an assembly instr...,To understand how a processor performs the add...,How does a processor turn an assembly instruct...,The process of turning an assembly instruction...,0.811062,0.230769
9,How do protons overcome their repulsion and co...,"When protons are initially far apart, they pos...",How do protons manage to overcome their mutual...,You're right to be puzzled by the fact that pr...,0.679870,0.264000


In [19]:
evaluate(model, tokenizer, prompts, f"{CHECKPOINT_DIR}/eval.csv")

,original_prompt,expected_answer,modified_prompt,actual_output,sbert_cosine_sim,rouge_l_recall
0,Discuss the efforts of the United Nations to a...,The United Nations is a global organization wi...,Describe the United Nations’ efforts aimed at ...,The United Nations has undertaken various effo...,0.785226,0.333333
1,"Why is a wing necessary for flight, given that...",The wing serves a crucial role in flight effic...,Why are wings essential for flight if engines ...,Wings are not only essential for flight; they ...,0.479776,0.202429
2,How can I plot a circle with matplotlib.pyplot...,The circle object that you created using `plt....,How can I draw a circle in matplotlib using on...,To draw a circle in matplotlib using only its ...,0.668165,0.214953
3,Discuss two practical and cost-effective ways ...,Solution 1: Implementing Renewable Energy Sour...,Examine two practical and affordable strategie...,Two practical and affordable strategies for en...,0.651768,0.215347
4,Explain the chain of events leading to the sig...,1. King John's rule: King John of England face...,Outline the sequence of events that led to the...,1. King John faced legal issues in England in ...,0.632545,0.293233
5,How is stellar parallax measured?,Stellar parallax is measured by observing the ...,How does one measure stellar parallax?,Stellar parallax is a method of measuring dist...,0.844923,0.295858
6,"What is bacterial transformation, and how does...",Bacterial transformation is a genetic modifica...,"What is bacterial transformation, and through ...",Bacterial transformation involves the acquisit...,0.858031,0.240437
7,What triggers the release of neurotransmitters?,"The axon is a long, thin projection that exten...",What causes neurons to release neurotransmitters?,Neurons release neurotransmitters primarily th...,0.781318,0.136364
8,How does a processor execute an assembly instr...,To understand how a processor performs the add...,How does a processor turn an assembly instruct...,A processor executes assembly instructions as ...,0.719354,0.167224
9,How do protons overcome their repulsion and co...,"When protons are initially far apart, they pos...",How do protons manage to overcome their mutual...,Protons can overcome their mutual repulsion be...,0.694558,0.224000


## Saving

In [20]:
model.save_pretrained_merged("model_full_16bit", tokenizer, save_method = "merged_16bit")
tokenizer.save_pretrained("model_full_16bit")

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/2 [00:00<?, ?it/s]

...

Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [01:46<00:00, 53.35s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [01:42<00:00, 51.21s/it]


Unsloth: Merge process complete. Saved to `/content/model_full_16bit`


('model_full_16bit/tokenizer_config.json',
 'model_full_16bit/special_tokens_map.json',
 'model_full_16bit/chat_template.jinja',
 'model_full_16bit/tokenizer.json')

In [21]:
!apt-get update
!apt-get install pciutils build-essential cmake curl libcurl4-openssl-dev -y
if not os.path.isdir("llama.cpp"):
    !git clone https://github.com/ggerganov/llama.cpp.git
else:
    print("llama.cpp already present.")
!pip install -U "transformers" "huggingface_hub"
!mkdir -p llama.cpp/build && cd llama.cpp/build && cmake .. && cmake --build . --config Release --target llama-quantize llama-cli llama-gguf-split llama-mtmd-cli

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,201 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,842 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: 

In [22]:
gguf_f16_file_name = "/content/Model-F16.gguf"
gguf_q4km_file_name = f"{CHECKPOINT_DIR}/Model-Q4_K_M.gguf"
!python llama.cpp/convert_hf_to_gguf.py ./model_full_16bit/ --outfile {gguf_f16_file_name} --outtype f16
!cd llama.cpp/build/bin && ./llama-quantize {gguf_f16_file_name} {gguf_q4km_file_name} q4_k_m

INFO:hf-to-gguf:Loading model: model_full_16bit
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: indexing model part 'model-00001-of-00002.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00002-of-00002.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> F16, shape = {3072, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> F16, shape = {8192, 3072}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_norm.